## Innio Myplant Fleet Analysis


In [ ]:
import arrow; print(arrow.now('Europe/Vienna').format('DD.MM.YYYY - HH:mm'))

In [ ]:
import dmyplant2
import json
import pandas as pd
import numpy as np
from tabulate import tabulate
from pprint import pprint as pp
import sys

In [ ]:
dmyplant2.cred()            # get store credentials
mp = dmyplant2.MyPlant(0)   # connect to myplant and store Request Session for later requests

In [ ]:
# fetch_installed_base(self,fields, properties, dataItems):
def get_installed_base():
    fields = ['serialNumber']

    properties =  [
        'Design Number','Engine Type','Engine Version','Engine Series','Engine ID',
        'Control System Type',
        'Country','IB Site Name','Commissioning Date','IB Unit Commissioning Date','Contract.Warranty Start Date', 'Contract.Warranty End Date','IB Status',
        'IB NOX', 'IB Frequency', 'IB Item Description Engine'
        ]

    dataItems = ['OperationalCondition','Module_Vers_HalIO','starts_oph_ratio','startup_counter',
    'shutdown_counter','Count_OpHour','Power_PowerNominal','Para_Speed_Nominal'
    ]
    fleet = mp.fetch_installed_base(fields, properties, dataItems, limit = None)
    fleet.to_pickle('Installed_base.pkl')
    return fleet

download_from_myplant = True
if download_from_myplant:
    fleet = get_installed_base()
else:
    try:
        fleet = pd.read_pickle('Installed_base.pkl')
    except:
        fleet = get_installed_base()

In [ ]:
ffleet = fleet[
    #~fleet.OperationalCondition.isin(['Never Connected','Decommissioned','No Contact']) &
    #~fleet['Engine Type'].isnull() &
    fleet['Engine Type'].isin(['624'])
    #~dassets['Engine Version'].isin(['G01','G02','G09','G11','G111','G12','G375'])  &
    #fleet['Engine Series'].isin(['6'])
    ].copy().reset_index()

#ffleet.groupby( "Engine Type")['Count_OpHour'].describe().fillna(0).astype(int)
ffleet[['Engine Type','Count_OpHour','OperationalCondition','Commissioning Date']].agg({
    'Engine Type':['count'],
    'Count_OpHour':['count','min','max','median','mean'],
    'OperationalCondition': ['count']
})

In [ ]:
ffleet[ffleet['Count_OpHour'] > 30000.0][['Commissioning Date','Count_OpHour','Engine Version','IB Site Name']].count()

In [ ]:
fffleet = fleet[
    #~fleet.OperationalCondition.isin(['Never Connected','Decommissioned','No Contact']) &
    ~fleet.OperationalCondition.isin(['Decommissioned']) &
    ~fleet['Commissioning Date'].isin([np.NaN,' ']) &
    #~fleet['Engine Type'].isnull() &
    fleet['Engine Type'].isin(['624'])
    #~dassets['Engine Version'].isin(['G01','G02','G09','G11','G111','G12','G375'])  &
    #fleet['Engine Series'].isin(['6'])
    ].copy().reset_index()
sffleet=fffleet.sort_values(by = "Commissioning Date",ascending=True)

dset = ['Commissioning Date','Count_OpHour','Engine Version','IB Site Name']

In [ ]:
sffleet[dset].head(15)

In [ ]:
sffleet[dset].tail(15)

In [ ]:
sffleet[sffleet['Count_OpHour'] > 0.0].hist(column=['Count_OpHour','startup_counter','Power_PowerNominal'],bins=50,figsize=(16,8));

In [ ]:
rpfleet = sffleet[sffleet['IB Item Description Engine'].fillna('').str.contains('Repairs')].copy()
rpfleet

In [ ]:
sffleet[dset].tail(15)

In [ ]:
# filter installed base
ffleet = fleet[
    fleet.OperationalCondition.isin(['Decommissioned']) &
    #~fleet['Engine Type'].isnull() &
    fleet['Engine Type'].isin(['624']) &   
    #~fleet['Engine Version'].isin(['G01','G02','G09','G11','G111','G12','G375'])  &
    fleet['Engine Series'].isin(['6'])
    ].copy().reset_index()

ffleet.groupby( "Engine Type")[['Count_OpHour']].describe()

In [ ]:
dfleet=fleet.groupby('Engine Type').head(1000000)
#dfleet.pivot_table(index='Engine Version', columns='Engine Type', values='Count_OpHour', aggfunc=['count','mean','max','sum'], margins=True).round(0).fillna('')
dfleet.pivot_table(index='Engine Series', columns='Engine Type', values='serialNumber', aggfunc=['count'], margins=False).fillna('')
#round(0).fillna(0).astype(int)